# 2024C1 SMI saxs films trexs plotting notebook for SMI

In [ ]:
# !pip install -e /nsls2/users/alevin/repos/PyHyperScattering  # to use pip to install via directory

## Imports

In [ ]:
import PyHyperScattering as phs
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import xarray as xr
import dask.array as da
from tqdm.auto import tqdm 
import subprocess
import io
import gc
print(phs.__version__)

In [ ]:
# import zarr

## Define paths, load xarrays, define functions

In [ ]:
# Define paths
propPath = pathlib.Path('/nsls2/data/smi/proposals/2024-1/pass-314903')
outPath = propPath.joinpath('processed_data/trexs_plots')
sampleZarrsPath = propPath.joinpath('processed_data/zarrs/saxs_core_films_trexs_sample_zarrs_v2')

# rawPaths = propPath.joinpath('raw_04')
# waxsPath = rawPaths.joinpath('900KW')

In [ ]:
sampleZarrsPath.exists()

In [ ]:
unique_sample_names = sorted(set(['_'.join(f.name.split('_')[1:3]) for f in sampleZarrsPath.glob('*')]))
unique_sample_names

In [ ]:
recip_DS_rows = []
caked_DS_rows = []
for sample_name in tqdm(unique_sample_names):
    sample_zarrs = sorted(sampleZarrsPath.glob(f'*{sample_name}*'))
    # display(sorted([f.name for f in sample_zarrs]))
    
    samp_recip_DS_rows = []
    samp_caked_DS_rows = []
    for sample_zarr in sample_zarrs:
        if 'recip_' in sample_zarr.name:
            recip_DS = xr.open_zarr(sample_zarr)
            samp_recip_DS_rows.append(recip_DS)
        elif 'caked_' in sample_zarr.name:
            caked_DS = xr.open_zarr(sample_zarr)
            samp_caked_DS_rows.append(caked_DS)
            
    recip_DS = xr.concat(samp_recip_DS_rows, 'theta')
    recip_DS_rows.append(recip_DS)
    
    caked_DS = xr.concat(samp_caked_DS_rows, 'theta')
    caked_DS_rows.append(caked_DS)
    
recip_DS = xr.concat(recip_DS_rows, 'sample_name')
caked_DS = xr.concat(caked_DS_rows, 'sample_name')

In [ ]:
recip_DS = recip_DS.chunk({'sample_name':1, 'pix_y': 3129, 'pix_x': 2943, 'energy':63,})
caked_DS = caked_DS.chunk({'sample_name':1, 'index_y':500,'index_x':500,'energy':63})

In [ ]:
recip_DS = recip_DS.swap_dims({'pix_y':'q_y', 'pix_x':'q_x'})
caked_DS = caked_DS.swap_dims({'index_y':'chi', 'index_x':'q_r'})
caked_DS

In [ ]:
def make_para_perp_DAs(DS, sample_name, theta=90, chi_width=90):
    # select dataarray to plot
    DA = DS.sel(sample_name=sample_name)['flatfield_corr']
    sel_DA = DA.sel(theta=theta)

    # calculate ISI dataarrays
    para_DA = sel_DA.rsoxs.slice_chi(0, chi_width=(chi_width/2))
    perp_DA = sel_DA.rsoxs.slice_chi(-90, chi_width=(chi_width/2))
        
    return para_DA, perp_DA

In [ ]:
cmap = plt.cm.turbo.copy()
cmap.set_bad(cmap.get_under())

In [ ]:
# recip_DS.to_zarr(outPath.joinpath('zarrs', f'recip_waxs_trexs_night2_standard.zarrs'), mode='w')

In [ ]:
# energy = 2470

# for sample_name in sample_names:
#     sliced_DA = recip_DS.sel(sample_name=sample_name).sel(energy=energy, method='nearest')['flatfield_corr']

#     cmin = sliced_DA.compute().quantile(0.15)
#     cmax = sliced_DA.compute().quantile(0.995)
#     ax = sliced_DA.plot.imshow(norm=plt.Normalize(cmin, cmax), cmap=plt.cm.turbo, x='q_x', y='q_y')
#     ax.axes.set(title=f'{sample_name}: Energy = {energy}')
#     plt.show()
#     plt.close('all')

## Detector movies

### recip

In [ ]:
float(da.percentile(sliced_DA.data.flatten(), 99))

In [ ]:
arr = sliced_DA[::3,::3].compute()

In [ ]:
# Select Dataset
DS = recip_DS


# Select Plotting Parameters
energy = 2477.2

# Select DataArray
for sample_name in tqdm(unique_sample_names[:]):
    for theta in [90]:
        DA = DS.sel(sample_name=sample_name, theta=theta)['flatfield_corr']

        # Plot
        sliced_DA = DA.sel(energy=energy,method='nearest')
        # cmin = float(da.percentile(sliced_DA.mean('q_y').data, 10))
        # cmax = float(da.percentile(sliced_DA.mean('q_y').data, 99.5))
        cmin = float(sliced_DA[::10, ::10].compute().quantile(0.15))
        cmax = float(sliced_DA[::10, ::10].compute().quantile(0.995))

        ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=plt.Normalize(cmin,cmax))
        ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
        ax.figure.set_tight_layout(True)
        ax.axes.set(aspect='equal', title=f'{sample_name}, $\\theta$ = {theta}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
        ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=12)
        plt.show()
        plt.close('all')

In [ ]:
unique_sample_names[7:]

In [ ]:
# Select Dataset
DS = recip_DS

# Select DataArray
for sample_name in tqdm(unique_sample_names[7:]):
    for theta in [90, 55, 35]:
        DA = DS.sel(sample_name=sample_name, theta=theta)['flatfield_corr']
        cmin = float(DA.compute().quantile(0.15))
        cmax = float(DA.compute().quantile(0.995))

        savePath = outPath.joinpath('saxs_core_films_trexs_plots_v2/recip_detector_movies_v1')
        savePath.mkdir(exist_ok=True)
        output_path = savePath.joinpath(f'{sample_name}_{theta}degth.mp4')

        # FFmpeg command. This is set up to accept data from the pipe and use it as input, with PNG format.
        # It will then output an H.264 encoded MP4 video.
        cmd = [
            'ffmpeg',
            '-y',  # Overwrite output file if it exists
            '-f', 'image2pipe',
            '-vcodec', 'png',
            '-r', '15',  # Frame rate
            '-i', '-',  # The input comes from a pipe
            '-vcodec', 'libx264',
            '-pix_fmt', 'yuv420p',
            '-crf', '17',  # Set the quality (lower is better, 17 is often considered visually lossless)
            str(output_path)
        ]

        # Start the subprocess
        proc = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Loop through the energy dimension and send frames to FFmpeg
        for i, energy in enumerate(tqdm(DA.energy.values, desc=f'Making the {sample_name} $\\theta$={theta}° movie')):
            # Make & customize plot
            sliced_DA = DA.sel(energy=energy, method='nearest')

            ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), x='q_x', y='q_y', cmap=cmap, norm=plt.Normalize(cmin,cmax))
            ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
            ax.figure.set_tight_layout(True)   
            ax.axes.set(aspect='equal', title=f'{sample_name}, $\\theta$ = {theta}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
            ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=12)

            # Save figure to a PNG buffer
            if i==0:
                ax.figure.savefig(savePath.joinpath(f'{sample_name}_{theta}degth.png'), dpi=120)

            buf = io.BytesIO()
            ax.figure.savefig(buf, format='png')
            buf.seek(0)

            # Write the PNG buffer data to the process
            proc.stdin.write(buf.getvalue())
            plt.close('all')

        # Finish the subprocess
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(f"Error: {err}")
            
        gc.collect()

### caked

In [ ]:
# Select Dataset
DS = caked_DS


# Select Plotting Parameters
energy = 2477
# chi_slice = slice(-150,50)
chi_slice = slice(None,None)

qr_slice = slice(0.015,0.085)
# qr_slice = slice(None,None)

# Select DataArray
# sample_name = 'PM6-Y6_3000_dSiN'
for sample_name in tqdm(unique_sample_names[:]):
    for theta in [90]:
        DA = DS.sel(sample_name=sample_name, theta=theta)['flatfield_corr']

        # Plot
        sliced_DA = DA.sel(energy=energy,method='nearest').sel(chi=chi_slice, q_r=qr_slice)
        cmin = float(sliced_DA.compute().quantile(0.1))
        cmax = float(sliced_DA.compute().quantile(0.99))

        ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), x='q_r', y='chi', cmap=cmap, norm=plt.Normalize(cmin,cmax),
                                   xscale='log')
        ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
        ax.figure.set_tight_layout(True)
        ax.axes.set(title=f'{sample_name}, $\\theta$ = {theta}°', xlabel='q$_r$ [$Å^{-1}$]', ylabel='$\\chi$ [°]')
        ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=12)
        plt.show()
        plt.close('all')

In [ ]:
outPath

In [ ]:
# Select Dataset
DS = caked_DS.copy()

# plotting parameters
# chi_slice = slice(-150,50)
chi_slice = slice(None,None)

qr_slice = slice(0.015,0.085)
# qr_slice = slice(None,None)

# Select DataArray
for sample_name in tqdm(unique_sample_names[:]):
    for theta in [90, 55, 35]:
        DA = DS.sel(sample_name=sample_name, theta=theta)['flatfield_corr']
        cmin = float(DA.compute().quantile(0.15))
        cmax = float(DA.compute().quantile(0.995))

        savePath = outPath.joinpath('saxs_core_films_trexs_plots_v2/caked_detector_movies_v1')
        savePath.mkdir(exist_ok=True)
        output_path = savePath.joinpath(f'{sample_name}_{theta}degth.mp4')

        # FFmpeg command. This is set up to accept data from the pipe and use it as input, with PNG format.
        # It will then output an H.264 encoded MP4 video.
        cmd = [
            'ffmpeg',
            '-y',  # Overwrite output file if it exists
            '-f', 'image2pipe',
            '-vcodec', 'png',
            '-r', '15',  # Frame rate
            '-i', '-',  # The input comes from a pipe
            '-vcodec', 'libx264',
            '-pix_fmt', 'yuv420p',
            '-crf', '17',  # Set the quality (lower is better, 17 is often considered visually lossless)
            str(output_path)
        ]

        # Start the subprocess
        proc = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Loop through the energy dimension and send frames to FFmpeg
        for i, energy in enumerate(tqdm(DA.energy.values, desc=f'Making the {sample_name} {theta}° movie')):
            # Make & customize plot
            sliced_DA = DA.sel(energy=energy,method='nearest').sel(chi=chi_slice, q_r=qr_slice)

            ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=plt.Normalize(cmin,cmax),
                                       xscale='log')
            ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
            ax.figure.set_tight_layout(True)   
            ax.axes.set(title=f'{sample_name}, $\\theta$ = {theta}°', xlabel='q$_r$ [$Å^{-1}$]', ylabel='$\\chi$ [°]')
            ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=12)

            # Save figure if first frame:
            if i==0:
                ax.figure.savefig(savePath.joinpath(f'{sample_name}_{theta}degth.png'), dpi=120)

            buf = io.BytesIO()
            ax.figure.savefig(buf, format='png')
            buf.seek(0)

            # Write the PNG buffer data to the process
            proc.stdin.write(buf.getvalue())
            plt.close('all')

        # Finish the subprocess
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(f"Error: {err}")
            
        gc.collect()

## Intensity plots

### 1. 2D intensity maps

In [ ]:
unique_sample_names

In [ ]:
# make selecan
DS = caked_DS.copy()

# make selection
q_slice = slice(0.015, 0.085)
chi_width = 90
energy_slice = slice(2470, 2485)


for sample_name in tqdm(unique_sample_names[:]):
    for theta in [90, 55, 35]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, theta, chi_width) 

        para_DA = para_DA.where(para_DA>0.01)  #.where(para_DA<(para_DA.max()*0.5))  #.interpolate_na(dim='chi')
        # para_DA.sel(energy=2477.2,method='nearest').sel(q_r=q_slice).plot.imshow(xscale='log', cmap=cmap)
        # plt.title('para')
        # plt.show()

        perp_DA = perp_DA.where(perp_DA>0.01)  #.where(perp_DA<(perp_DA.max()*0.5))  #.interpolate_na(dim='chi')
        # perp_DA = perp_DA - perp_sin90_DA * p
        # perp_DA.sel(energy=2477.2,method='nearest').sel(q_r=q_slice).plot.imshow(xscale='log', cmap=cmap)
        # plt.title('perp')
        # plt.show()
        # plt.close('all')

        # Plot
        fig, axs = plt.subplots(1, 2, figsize=(10,5))
        cmin = float(para_DA.compute().quantile(0.2))
        cmax = float(perp_DA.compute().quantile(0.999))

        para_slice = para_DA.mean('chi').sel(q_r=q_slice, energy=energy_slice)  # .plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
        perp_slice = perp_DA.mean('chi').sel(q_r=q_slice, energy=energy_slice)  # .plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

        para_slice.plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
        perp_slice.plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

        # Add colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
        cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        cbar.set_label(label='Intensity [arb. units]', labelpad=12, rotation=270)

        fig.suptitle(f'Intensity maps: {sample_name}, $\\theta$ = {theta}°, $\\chi$ width = {chi_width}°', fontsize=14)
        fig.set(tight_layout=True)

        axs[0].set(xscale='log', title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='Q [$Å^{-1}$]')
        axs[1].set(xscale='log', title='Perpendicular to $E_p$ ', ylabel=None, xlabel='Q [$Å^{-1}$]')

        savePath = outPath.joinpath('saxs_core_films_trexs_plots_v2/I_maps_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath( 
                    f'{sample_name}_theta-{theta}deg_{round(cmin/1e7)}cmin-{round(cmax/1e7)}cmax_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_energy{energy_slice.start}-{energy_slice.stop}.png'), 
                    dpi=120)

        # plt.show()
        plt.close('all')

In [ ]:
# a = para_EvsQ_avg.hvplot.line(groupby="energy", loglog=True)
# a * perp_EvsQ_avg.hvplot.line(groupby="energy")

### 2. 1D meaned intensity linecuts

In [ ]:
# make selection
DS = caked_DS.copy()

# make selection
q_slice = slice(0.015, 0.08)
chi_width = 90
# energy_slice = slice(2470, 2485)

multiplier=0.0014

for sample_name in tqdm(unique_sample_names[:]):
    for theta in [90, 55, 35]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, theta, chi_width) 

        para_DA = para_DA.where(para_DA>0.01)  #.where(para_DA<(para_DA.max()*multiplier))  #.interpolate_na(dim='chi')
        # para_DA.sel(energy=2477.2,method='nearest').sel(q_red=slice(0.05, 0.7)).plot.imshow()
        # plt.title('para')
        # plt.show()

        perp_DA = perp_DA.where(perp_DA>0.01)  #.where(perp_DA<(perp_DA.max()*4*multiplier))  #.interpolate_na(dim='chi')
        # perp_DA.sel(energy=2477.2,method='nearest').sel(q_red=slice(0.05, 0.7)).plot.imshow()
        # plt.title('perp')
        # plt.show()
        # plt.close('all')

        # Plot
        energies = para_DA.energy.sel(energy=[2445, 2470.2, 2472, 2474, 2476, 2477, 2478, 2484, 2550], method='nearest').data

        cmap = plt.cm.turbo.copy()
        colors = cmap(np.linspace(0, 1, len(energies)))

        fig, axs = plt.subplots(ncols=2,figsize=(10,4), tight_layout=True)

        for j, energy in enumerate(energies):
            (para_DA.sel(q_r=q_slice, energy=energy).mean('chi')
             .plot.line(ax=axs[0], color=colors[j], yscale='log', xscale='log', label=energy))
            (perp_DA.sel(q_r=q_slice, energy=energy).mean('chi')
             .plot.line(ax=axs[1], color=colors[j], yscale='log', xscale='log', label=energy))

        fig.suptitle(f'IvsQ, $\\theta$ = {theta}°, $\chi$ width = 90°: {sample_name}', x=0.47)

        axs[0].set(title=f'Parallel to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
        axs[1].set(title=f'Perpendicular to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
        axs[1].legend(title='Energy [eV]', loc=(1.05,0.1))

        savePath = outPath.joinpath('saxs_core_films_trexs_plots_v2/I_cuts_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath( 
                    f'{sample_name}_theta-{theta}deg_{round(cmin/1e7)}cmin-{round(cmax/1e7)}cmax_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_energy{energy_slice.start}-{energy_slice.stop}.png'), 
                    dpi=120)

        # plt.show()
        plt.close('all')

## ISI plots

In [ ]:
# DA = rsoxs_datasets['polar_carbon']['corr_intensity']
# DA.sel(sample_name='Y6_CB_3000', polarization=0, q=slice(0.001, 0.03)).sel(energy=398.8, method='nearest').plot(norm=LogNorm(1e9, 1e11), cmap=plt.cm.turbo)
# plt.show()

In [ ]:
outPath

In [ ]:
# make selection
DS = caked_DS.copy()

# make selection
q_slice = slice(0.015, 0.085)
chi_width = 90
energy_slice = slice(2470, 2485)

for sample_name in tqdm(unique_sample_names[:]):
    for theta in [90, 55, 35]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, theta, chi_width) 

        para_DA = para_DA.where(para_DA>0.01)  #.where(para_DA<(para_DA.max()*0.0007))  #.interpolate_na(dim='chi')
        # para_DA.sel(energy=2477.2,method='nearest').sel(q_r=q_slice).plot.imshow()
        # plt.title('para')
        # plt.show()

        perp_DA = perp_DA.where(perp_DA>0.01)  #.where(perp_DA<(perp_DA.max()*0.0039))  #.interpolate_na(dim='chi')
        # perp_DA.sel(energy=2477.2,method='nearest').sel(q_r=q_slice).plot.imshow()
        # plt.title('perp')
        # plt.show()
        # plt.close('all')


        # slice ISI data
        para_ISI = para_DA.sel(q_r=q_slice).mean('chi').mean('q_r')
        perp_ISI = perp_DA.sel(q_r=q_slice).mean('chi').mean('q_r')

        # plot
        fig, ax = plt.subplots(figsize=(6,4), dpi=120)

        para_ISI.sel(energy=energy_slice).plot.line(ax=ax, label='$\parallel$', yscale='log')
        ax2=ax.twinx()
        perp_ISI.sel(energy=energy_slice).plot.line(ax=ax2, label='$\perp$', yscale='log', color='C1')
        fig.suptitle(f'ISI: {sample_name}, $\\theta$ = {theta}°', fontsize=14, x=0.55)
        ax.set(title=f'Chi width = {chi_width}°, Q = ({q_slice.start}, {q_slice.stop}) ' + 'Å$^{-1}$', 
               xlabel='X-ray energy [eV]', ylabel='$\parallel$ Intensity [arb. units]')
        ax2.set(title=None, ylabel='$\perp$ Intensity [arb. units]')
        ax2.legend(loc='upper right', fontsize=14)
        ax.legend(loc='upper left', fontsize=14)
        ax.grid(axis='x')
        plt.subplots_adjust(top=0.86, bottom=0.2, left=0.2)

        savePath = outPath.joinpath('saxs_core_films_trexs_plots_v2/ISIs_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath( 
                    f'{sample_name}_theta-{theta}deg_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_energy{energy_slice.start}-{energy_slice.stop}.png'), 
                    dpi=120)

        # plt.show()
        plt.close('all')

## Anisotropy Ratio Plots

### 1. 2D AR Maps

In [ ]:
# caked_DS = caked_DS.swap_dims({'index_y':'chi'})

# q_r_coords = caked_DS.q_r.mean('energy')
# caked_DS = caked_DS.assign_coords({'q_red':('index_x', q_r_coords.data)}).swap_dims({'index_x':'q_red'})
# caked_DS

In [ ]:
sample_names

In [ ]:
DS = caked_DS.copy()

# make selection
q_slice = slice(0.015, 0.085)
chi_width = 90
energy_slice = slice(2470, 2485)

# para_sin35, perp_sin35 = make_para_perp_DAs(DS, 'BareSiN_01', 35, chi_width) 
# para_sin55, perp_sin55 = make_para_perp_DAs(DS, 'BareSiN_01', 55, chi_width) 
# para_sin90, perp_sin90 = make_para_perp_DAs(DS, 'BareSiN_01', 90, chi_width) 

for sample_name in tqdm(unique_sample_names[:]):
    for theta in [90, 55, 35]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, theta, chi_width) 

        para_DA = para_DA.where(para_DA>0.01)  #.where(para_DA<(para_DA.max()*0.9))  #.interpolate_na(dim='chi')
        # para_DA.sel(energy=2477.2,method='nearest').sel(q_red=slice(0.05, 0.7)).plot.imshow()
        # plt.title('para')
        # plt.show()

        perp_DA = perp_DA.where(perp_DA>0.01)  #.where(perp_DA<(perp_DA.max()*0.9))  #.interpolate_na(dim='chi')
        # perp_DA.sel(energy=2477.2,method='nearest').sel(q_red=slice(0.05, 0.7)).plot.imshow()
        # plt.title('perp')
        # plt.show()
        # plt.close('all')

        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        # Plot
        vlim = 0.5
        ax = ar_DA.sel(energy=energy_slice, q_r=q_slice).plot.pcolormesh(
            cmap=plt.cm.RdBu_r, figsize=(8,5), norm=plt.Normalize(-vlim, vlim), xscale='log')
        # ax = ar_DA.sel(energy=energy_slice, q_red=q_slice).plot.pcolormesh(figsize=(8,5))

        ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
        ax.axes.set(title=f'{sample_name}, $\\theta$ = {theta}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]')
        ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)


        savePath = outPath.joinpath('saxs_core_films_trexs_plots_v2/ar_maps_v1')
        savePath.mkdir(exist_ok=True)
        ax.figure.savefig(savePath.joinpath( 
                          f'{sample_name}_theta-{theta}deg_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_energy{energy_slice.start}-{energy_slice.stop}.png'), 
                          dpi=120)

        # plt.show()
        plt.close('all')

In [ ]:
outPath.joinpath('trexs_plots/ar_maps_v1')

### 2. 1D meaned AR linecuts

In [ ]:
# make selection
DS = caked_DS.copy()

# make selection
q_slice = slice(0.015, 0.085)
chi_width = 90
# energy_slice = slice(2470, 2485)

for sample_name in tqdm(unique_sample_names[:]):
    for theta in [90, 55, 35]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, theta, chi_width) 
        
        para_DA = para_DA.where(para_DA>0.4).where(para_DA<(para_DA.max()*0.5))  #.interpolate_na(dim='chi')
        # para_DA.sel(energy=2477.2,method='nearest').sel(q_red=slice(0.05, 0.7)).plot.imshow()
        # plt.title('para')
        # plt.show()
        
        perp_DA = perp_DA.where(perp_DA>0.4).where(perp_DA<(perp_DA.max()*0.5))  #.interpolate_na(dim='chi')
        # perp_DA.sel(energy=2477.2,method='nearest').sel(q_red=slice(0.05, 0.7)).plot.imshow()
        # plt.title('perp')
        # plt.show()
        # plt.close('all')
            
        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        # Plot   
        energies = para_DA.energy.sel(energy=[2445, 2470.2, 2472, 2474, 2476, 2477, 2478, 2484, 2550], method='nearest').data
        cmap = plt.cm.turbo.copy()
        colors = cmap(np.linspace(0, 1, len(energies)))        

        # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
        # cax = ax.inset_axes([1.03, 0, 0.03, 1])
        # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        # cbar.set_label(label=f'Time [seconds]', labelpad=14)
        # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

        fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

        for j, energy in enumerate(energies):
            (ar_DA.sel(energy=energy, q_r=q_slice).plot.line(ax=ax, color=colors[j], label=energy))

        fig.suptitle(f'Anisotropy Ratio Linecuts: {sample_name}', x=0.46)

        ax.set(title=f'Chi width = {chi_width}°, $\\theta$ = {theta}°', ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
        ax.legend(title='Energy [eV]', loc=(1.03,0.1))

        savePath = outPath.joinpath('saxs_core_films_trexs_plots_v2/ar_cuts_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath( 
                    f'{sample_name}_theta-{theta}deg_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_energy{energy_slice.start}-{energy_slice.stop}.png'), 
                    dpi=120)

        # plt.show()
        plt.close('all')

In [ ]:
ar_DA.hvplot.line(groupby="energy")